# Simple Example of Incremental Learning Agent w/Python + Keras (MultiClass Example)

Example of Incremental Learning Agent on Multiclass problem.

In [2]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

np.random.seed(4)

In [3]:
df = pd.read_csv("../datasets/iris.csv", header=None)
df = df.values
X = df[:,0:4].astype(float)
y = df[:,4]

In [4]:
# Encode class values
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# One hot encoding
dummy_y = np_utils.to_categorical(encoded_Y)

In [5]:
import hashlib
from keras.models import load_model

In [6]:
class IncrementalAgent:
    
    def __init__(self, name):
        # Identify agent
        self.name = name
        # Need to keep track of predictions for evaluation of the model
        self.prediction_records = {}
        # Create model
        self.model = Sequential()
        self.model.add(Dense(4, input_dim=4, kernel_initializer='normal', activation='relu'))
        self.model.add(Dense(3, kernel_initializer='normal', activation='sigmoid'))
        # Compile model
        self.model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        print("Model Initialized.")

    
    def train(self, feature_array, target, epoch_count, batch_count):
        target_to_array = np.array([target])
        print(target_to_array)
        self.model.fit(feature_array, target_to_array, epochs=epoch_count, batch_size=batch_count)
        self.model.save('{}.h5'.format(self.name))
        print("Model Trained.")
    
    
    def evaluate(self, feature_array, target_class):
        # Check if the features have previously been seen
        prediction_ref = hashlib.sha256(feature_array).hexdigest()
        if prediction_ref in self.prediction_records:
            print("Previous Predicted Class: {}".format(self.prediction_records[prediction_ref]))
            print("Actual Class: {}".format(target_class))
            if self.prediction_records[prediction_ref] == target_class:
                print("Prediction Correct!")
            else:
                print("Prediction Incorrect. Relearn...")
                self.train(feature_array, target, 5, 1)
                print("Retry Predict & Update Previous Prediction")
                self.predict(feature_array)
                
        else:
            print("First time seeing this sample.")
    

    def predict(self, feature_array):
        # Create ID for prediction
        prediction_ref = hashlib.sha256(feature_array).hexdigest()
        # Actual prediction
        probability_prediction = self.model.predict(feature_array)
        
        # Get Class Prediction
        prediction = probability_prediction.argmax() 
        
        # Add Prediction to Records
        self.prediction_records[prediction_ref] = prediction
        
        print("Predicted Class: {}".format(prediction))

In [7]:
from sklearn.model_selection import train_test_split

# Split for training and evaluation
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33)

In [8]:
sample = np.array([X_train[2]])
target = np.array([dummy_y[2]])[0]

In [9]:
agent1 = IncrementalAgent("multiclass_agent")

Model Initialized.


In [10]:
agent1.train(sample, target, 5, 1)

[[1. 0. 0.]]
Epoch 1/5
1/1 [==============================] - 0s 190ms/step - loss: 1.0970 - acc: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 2ms/step - loss: 1.0941 - acc: 0.0000e+00
Epoch 3/5
1/1 [==============================] - 0s 2ms/step - loss: 1.0912 - acc: 0.0000e+00
Epoch 4/5
1/1 [==============================] - 0s 2ms/step - loss: 1.0882 - acc: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 2ms/step - loss: 1.0854 - acc: 1.0000
Model Trained.


In [11]:
next_sample = np.array([X_train[3]])
next_target = np.array([dummy_y[3]])[0]

In [12]:
next_target = next_target.argmax()

In [13]:
print(next_target)

0


In [14]:
agent1.predict(next_sample)

Predicted Class: 0


In [15]:
agent1.evaluate(next_sample, next_target)

Previous Predicted Class: 0
Actual Class: 0
Prediction Correct!
